In [1]:
import numpy as np
from scipy import stats
from scipy.spatial.distance import pdist, squareform
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
import seaborn as sns
from IPython.display import display
pd.set_option('max_rows', 5)
pd.set_option('max_columns', 9)
%matplotlib inline

# k近傍法 (k-nearest neighbors)
---
特徴空間内で距離的に近い教師データに基づいて分類するモデル。

In [2]:
iris = sns.load_dataset('iris')[['sepal_length', 'sepal_width', 'species']]
iris['species'] = pd.Categorical(iris['species'])
print('iris')
display(iris)

iris


,sepal_length,sepal_width,species
0,5.1,3.5,setosa
1,4.9,3.0,setosa
...,...,...,...
148,6.2,3.4,virginica
149,5.9,3.0,virginica


In [3]:
from my_functions.knn import algorithm
algorithm.show(iris.iloc[:, :2].values, iris['species'].cat.codes)

interactive(children=(IntSlider(value=3, continuous_update=False, description='k', max=10, min=1), Output()), …

## 仕組み
---
- 分類したいデータから特徴空間内での距離の近いk個の教師データの多数決で分類を決定
- 訓練データからパラメータを学習せず、訓練データを暗記する怠惰学習(lazy learner)の代表例
- 学習にかかるコストはないが、訓練データが増えると記憶容量・計算量が問題になる
- 特徴が増えるに従って、次元の呪い (計算量が爆発的に増えたり、精度が極端に悪化すること) に陥りやすい

練習問題

---
`wine`データセットのサンプル間距離を算出 (`scipy.spatial.distance.pdist`と`scipy.spatial.distance.squareform`を使用) し、それぞれのサンプルから最も近い 3 サンプルの`target`ラベルに基づいて、予測クラスを作成する。

In [4]:
loader = load_wine()
wine = pd.DataFrame(np.column_stack([loader.data, loader.target]),
                    columns=loader.feature_names + ['target'])
print('wine')
display(wine)

wine


,alcohol,malic_acid,ash,alcalinity_of_ash,...,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,...,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,...,1.05,3.40,1050.0,0.0
...,...,...,...,...,...,...,...,...,...
176,13.17,2.59,2.37,20.0,...,0.60,1.62,840.0,2.0
177,14.13,4.10,2.74,24.5,...,0.61,1.60,560.0,2.0


In [5]:
help(pdist)

Help on function pdist in module scipy.spatial.distance:

pdist(X, metric='euclidean', *args, **kwargs)
    Pairwise distances between observations in n-dimensional space.
    
    See Notes for common calling conventions.
    
    Parameters
    ----------
    X : ndarray
        An m by n array of m original observations in an
        n-dimensional space.
    metric : str or function, optional
        The distance metric to use. The distance function can
        be 'braycurtis', 'canberra', 'chebyshev', 'cityblock',
        'correlation', 'cosine', 'dice', 'euclidean', 'hamming',
        'jaccard', 'jensenshannon', 'kulsinski', 'mahalanobis', 'matching',
        'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean',
        'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule'.
    *args : tuple. Deprecated.
        Additional arguments should be passed as keyword arguments
    **kwargs : dict, optional
        Extra arguments to `metric`: refer to each metric documentation for a

In [6]:
help(squareform)

Help on function squareform in module scipy.spatial.distance:

squareform(X, force='no', checks=True)
    Convert a vector-form distance vector to a square-form distance
    matrix, and vice-versa.
    
    Parameters
    ----------
    X : ndarray
        Either a condensed or redundant distance matrix.
    force : str, optional
        As with MATLAB(TM), if force is equal to ``'tovector'`` or
        ``'tomatrix'``, the input will be treated as a distance matrix or
        distance vector respectively.
    checks : bool, optional
        If set to False, no checks will be made for matrix
        symmetry nor zero diagonals. This is useful if it is known that
        ``X - X.T1`` is small and ``diag(X)`` is close to zero.
        These values are ignored any way so they do not disrupt the
        squareform transformation.
    
    Returns
    -------
    Y : ndarray
        If a condensed distance matrix is passed, a redundant one is
        returned, or if a redundant one is passed

解答例

---

In [7]:
dist_matrix = squareform(pdist(wine.iloc[:, :-1].values))
nearests3 = pd.DataFrame(np.argsort(dist_matrix, axis=1)[:, :3])
pred = stats.mode(nearests3.applymap(lambda i: wine.iloc[i, -1]),
                  axis=1).mode[:, 0]

# scikit-learnの結果と比較
knn = KNeighborsClassifier(n_neighbors=3).fit(wine.iloc[:, :-1], wine['target'])
print(f'正解率 : {sum(pred == knn.predict(wine.iloc[:, :-1])) / wine.index.size}')

正解率 : 1.0


---

### 次元の呪い (高次元空間内の距離)
---
$1$ 次元空間 (直線) 上では変数の値が $1$ 増えると、その間の距離は $1$ 。  
$2$ 次元空間 (平面) 上では変数の値が $1$ ずつ増えると、その間の距離は $\sqrt{2}$ 。 (元の点から距離 $1$ の点の集合は半径 $1$ の円)  
$3$ 次元空間 (空間) 上では変数の値が $1$ ずつ増えると、その間の距離は $\sqrt{3}$ 。 (元の点から距離 $1$ の点の集合は半径 $1$ の球)  
$\vdots$  
$n$ 次元空間 (超空間) 上では変数の値が $1$ ずつ増えると、その間の距離は $\sqrt{n}$ 。

以上のように、変数が増えるにしたがって、それぞれの変数のわずかな変化によってサンプル同士がいずれのサンプルからも遠くなってしまい、近いサンプルという考え方の妥当性が低くなる。これが次元の呪いの原因の一つ。

## Pythonでのk近傍法の実行方法
---
`sklearn.neighbors.KNeighborsClassifier`を使用する。使用できる距離は[`sklearn.neighbors.DistanceMetric`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html)を参照。

In [8]:
help(KNeighborsClassifier)

Help on class KNeighborsClassifier in module sklearn.neighbors._classification:

class KNeighborsClassifier(sklearn.neighbors._base.NeighborsBase, sklearn.neighbors._base.KNeighborsMixin, sklearn.neighbors._base.SupervisedIntegerMixin, sklearn.base.ClassifierMixin)
 |  KNeighborsClassifier(n_neighbors=5, *, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None, **kwargs)
 |  
 |  Classifier implementing the k-nearest neighbors vote.
 |  
 |  Read more in the :ref:`User Guide <classification>`.
 |  
 |  Parameters
 |  ----------
 |  n_neighbors : int, default=5
 |      Number of neighbors to use by default for :meth:`kneighbors` queries.
 |  
 |  weights : {'uniform', 'distance'} or callable, default='uniform'
 |      weight function used in prediction.  Possible values:
 |  
 |      - 'uniform' : uniform weights.  All points in each neighborhood
 |        are weighted equally.
 |      - 'distance' : weight points by the inverse of t

In [9]:
help(DistanceMetric)

Help on class DistanceMetric in module sklearn.neighbors._dist_metrics:

class DistanceMetric(builtins.object)
 |  DistanceMetric class
 |  
 |  This class provides a uniform interface to fast distance metric
 |  functions.  The various metrics can be accessed via the :meth:`get_metric`
 |  class method and the metric string identifier (see below).
 |  
 |  Examples
 |  --------
 |  >>> from sklearn.neighbors import DistanceMetric
 |  >>> dist = DistanceMetric.get_metric('euclidean')
 |  >>> X = [[0, 1, 2],
 |           [3, 4, 5]]
 |  >>> dist.pairwise(X)
 |  array([[ 0.        ,  5.19615242],
 |         [ 5.19615242,  0.        ]])
 |  
 |  Available Metrics
 |  
 |  The following lists the string metric identifiers and the associated
 |  distance metric classes:
 |  
 |  **Metrics intended for real-valued vector spaces:**
 |  
 |  ==============  ====================  ========  ===============================
 |  identifier      class name            args      distance function
 |  -

## 推薦図書
---
- [見て試してわかる機械学習アルゴリズムの仕組み 機械学習図鑑](https://www.amazon.co.jp/%E8%A6%8B%E3%81%A6%E8%A9%A6%E3%81%97%E3%81%A6%E3%82%8F%E3%81%8B%E3%82%8B%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%82%A2%E3%83%AB%E3%82%B4%E3%83%AA%E3%82%BA%E3%83%A0%E3%81%AE%E4%BB%95%E7%B5%84%E3%81%BF-%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E5%9B%B3%E9%91%91-%E7%A7%8B%E5%BA%AD-%E4%BC%B8%E4%B9%9F/dp/4798155659/)
- [Python 機械学習プログラミング 達人データサイエンティストによる理論と実践](https://www.amazon.co.jp/Python-%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0-%E9%81%94%E4%BA%BA%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%83%86%E3%82%A3%E3%82%B9%E3%83%88%E3%81%AB%E3%82%88%E3%82%8B%E7%90%86%E8%AB%96%E3%81%A8%E5%AE%9F%E8%B7%B5-impress-gear/dp/4295003379/)